# FLIPKART FINAL AUTOMATED SCRAPING ALGORITHM

In [3]:
#Loading Required Liabraries
import re
import csv 
import requests
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions


#user defined function for accessing the product review page number URL and automating the page movement for fetchng the reviews
def get_url(link):
    template = link
    #add term query to url
    url = template.format(link)
    
    #add page query placeholder
    url += '&page={}'
    return url

#user defined function for extractig target data from the Amazon review webpage
def extract_record(item):
    """Extract and Return data from a single record"""
    
    
    ##Reviewer Name
    names = item.find('p', {'class':'_2sc7ZR _2V5EHH'}).text
    
    try:
        #Review Title
        title = item.find('p', {'class':'_2-N8zT'}).text.strip()
        review_title =title.lstrip('\n')
        review_title =title.rstrip('\n')
    except AttributeError:
        title = ''
    try:
        #Premlink
        premlink = item.find('div', {'class':'_1e9_Zu'}).text.strip()
    except AttributeError:
        premlink = ''
    
    try:
        #Review location 
        location = item.find('p', {'class' : '_2mcZGG'}).text.strip('\n')
        locations = re.sub(r"Certified Buyer", "",location)
        locations = locations.lstrip(',')
    except AttributeError:
        locations = ''
    
    try:
        #user help
        user_help = item.find('span', {'class':'_3c3Px5'}).text.strip()
    except AttributeError:
        #help_vote = '0'
        user_help = '0'
    try:
        #user not helpful
        user_nothelp = item.find('div', {'class':'pkR4jH'}).text.strip()
    except AttributeError:
        user_nothelp = '0'
    
    try:
        #Review date
        review_date = item.find('div', {'class':'row _3n8db9'}).text.strip()
        review_date = re.sub(r"Certified Buyer,", "",review_date)
        review_date = re.sub(r"Certified Buyer", "",review_date)
        review_date = re.sub(r"PermalinkReport Abuse", "",review_date)
        review_date = review_date.replace(names,"")
        review_date = review_date.replace(locations,"")
        a= len(user_help)
        b= len(user_nothelp)
        c= len(review_date)
        d= c-(a+b)
        review_date = review_date[:d]
    
    except AttributeError:
        review_date = ''    

        
    try:
        #Rating 
        rating = item.find('div', {'class':'_3LWZlK'}).text
        #rating_received = rating[0:3]
        #total_rating = rating[11]
    except AttributeError:
        #rating_received = ''
        #total_rating = ''
        rating = ''
        
    try:
        #Review Content 
        
        review = item.find('div', {'class' : 't-ZTKy'}).text.strip('\n ')
        review = re.sub(r"READ MORE", "",review)
    except AttributeError:
        review = ''
        
        
    result = (names, review_title,review_date,locations, rating,review,user_help)
    return result

#user defined main function for performing web scraping of reviews from the amazon webpage
def main(link):
    """Run Main Program Routine"""
    webdriver_location = "C:/Users/sarth/.wdm/drivers/edgedriver/win64/89.0.774.68/msedgedriver.exe"
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options=options, executable_path=webdriver_location)

    records = []
    url = get_url(link)
    
    #loop for getting the url of next review page along with accessing that page using beautiful soup
    for page in range(1,10):
        
        driver.get(url.format(page))
        soups = BeautifulSoup(driver.page_source, "html.parser")
        result = soups.find_all('div', {'class' : 'col _2wzgFH K0kLPL'})
        l = len(result)
        for i in range(1,l):
            #driver.implicitly_wait(10)
            try:
                driver.find_element_by_class_name('_1BWGvX').click()
            except:
                pass
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = soup.find_all('div', {'class' : 'col _2wzgFH K0kLPL'})
    
    #creating a loop for extracting the target data from each of the user review card
        for item in results:
            record = extract_record(item)
            if record:
                 records.append(record)
    
    driver.close()
    
            #save data to csv file
    with open('IAFS24XA3T4C_Flipkart.csv','w', newline='',encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Reviewer Name','Review Title','Review Date','Review Location', 'Rating Received', 'Review_Content', 'UP Vote'])
        writer.writerows(records)


In [4]:
#calling the main function with passing the link of first review page for a product
main('https://www.flipkart.com/ifb-2-ton-3-star-split-ac-white/product-reviews/itmf2h87c7bggf2z?pid=ACNFU3XZDFNVYUF2&lid=LSTACNFU3XZDFNVYUF2LYMF3L&marketplace=FLIPKART')